# AI Powered Query Profile and Recommendations Notebook
This notebook takes query profile data from the Snowflake get_query_operator_stats function and uses Snowflake's Cortex-powered SQL functions to describe the query execution plan, identify plan bottlenecks, and propose query and workload optimizations that might result in improved performance.

## Before Starting
Go to the query history page in Snowflake, find the query you are interested in, and copy the query_id. In the rest of this notebook you will need to replace "<query_id>" in the "Notebook Settings" panel with the copied query_id from query history.

In [ ]:
-- NOTEBOOK SETTINGS PANEL
-- Sets the query ID for your session

SET query_id='paste-query-id-here';

-- Sets the LLM selection for the model to use in the COMPLETE function

SET cortex_llm='claude-3-5-sonnet';

-- List of possible values for COMPLETE function
-- claude-3-5-sonnet gemma-7b jamba-1.5-mini jamba-1.5-large jamba-instruct llama2-70b-chat llama3-8b llama3-70b llama3.1-8b llama3.1-70b llama3.1-405b llama3.2-1b llama3.2-3b mistral-large mistral-large2 mistral-7b mixtral-8x7b reka-core reka-flash snowflake-arctic snowflake-llama-3.1-405b snowflake-llama-3.3-70b

-- See docs for regional availability: https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#label-cortex-llm-availability


In [ ]:
-- TABULAR OUTPUT PANEL
-- This panel generates the standard tabular output of get_query_operator_stats

SELECT * FROM TABLE(GET_QUERY_OPERATOR_STATS('01bb18ee-0004-c37b-0004-2477003ffa7e'));

In [ ]:
## AI POWERED PROFILE OUTPUT PANEL
## This panel generates an AI powered description of get_query_operator_stats

from snowflake.snowpark.context import get_active_session
session = get_active_session()

df = session.sql("""
WITH payload
     AS (SELECT Concat('STEP:', step_id, '|OPERATOR_ID', operator_id,
                '|OPERATOR_STATS',
                           operator_statistics, '|OPERATOR_TIME_DETAILS',
                           execution_time_breakdown, '|OPERATOR_TIME_ATTRIBUTES'
                ,
                           operator_attributes) AS operator_stats
         FROM   TABLE (Get_query_operator_stats(
                       '01bb18ee-0004-c37b-0004-2477003ffa7e')))
SELECT snowflake.cortex.Complete($cortex_llm, Concat(
'Describe in detail in spanish the query plan, time taken in each step, and operators from this snowflake query_opeartor_stats oayload: <payload>'
, Listagg(operator_stats, ', ')
within GROUP (ORDER BY operator_stats), '</payload>'))
FROM   payload; 
""").to_pandas()

print(df.iloc[0,0])

In [ ]:
## AI POWERED RECOMMENDATIONS OUTPUT PANEL
## This panel generates an AI powered recommendations based on the query profile

from snowflake.snowpark.context import get_active_session
session = get_active_session()

df = session.sql("""
WITH payload
     AS (SELECT Concat('STEP:', step_id, '|OPERATOR_ID', operator_id,
                '|OPERATOR_STATS',
                           operator_statistics, '|OPERATOR_TIME_DETAILS',
                           execution_time_breakdown, '|OPERATOR_TIME_ATTRIBUTES'
                ,
                           operator_attributes) AS operator_stats
         FROM   TABLE (Get_query_operator_stats(
                       '01bb18ee-0004-c37b-0004-2477003ffa7e')))
SELECT snowflake.cortex.Complete($cortex_llm, Concat(
'How could I improve the performance of this query, based on the payload from this snowflake query_opeartor_stats response. Specifically, how might I use clustering, Query Acceleration Service, or Search Optimization. Also, could I rewrite the query - make sure to use Snowflake compatible SQl syntax? Describe the answer in Spanish: <payload>'
, Listagg(operator_stats, ', ')
within GROUP (ORDER BY operator_stats), '</payload>'))
FROM   payload;
""").to_pandas()

print(df.iloc[0,0])